In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, classification_report
from collections import Counter

In [ ]:
df_data=pd.read_csv('data.csv', delimiter=',')
df_data

,Unnamed: 0,store_id,order_id,profit,delivery_distance,order_start_prepare,planned_prep_time,order_ready,region_id,status_id,...,items_count,order_price,day_of_week,hour_start,minutes_start,hour_start_sin,hour_start_cos,minutes_start_sin,minutes_start_cos,prepared_on_time
0,0,5014,20562761,45.5040,2205,2024-09-05 15:25:33,15.0,2024-09-05 15:32:00,683,F,...,2,30.49,Четверг,15,25,-0.707107,-0.707107,0.500000,-0.866025,1
1,1,3856,20562753,30.3360,1368,2024-09-05 15:25:19,10.0,2024-09-05 15:35:50,683,F,...,2,240.78,Четверг,15,25,-0.707107,-0.707107,0.500000,-0.866025,1
2,2,10825,20562723,43.6080,459,2024-09-05 15:23:44,20.0,2024-09-05 15:38:38,683,F,...,1,85.99,Четверг,15,23,-0.707107,-0.707107,0.669131,-0.743145,1
3,3,5640,20562673,32.8640,2611,2024-09-05 15:19:05,20.0,2024-09-05 15:35:25,709,F,...,2,24.68,Четверг,15,19,-0.707107,-0.707107,0.913545,-0.406737,1
4,4,7283,20562665,11.0600,1041,2024-09-05 15:14:58,15.0,2024-09-05 15:35:33,707,F,...,1,14.02,Четверг,15,14,-0.707107,-0.707107,0.994522,0.104528,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7462,7462,7557,20535845,50.1650,8046,2024-09-03 15:03:01,35.0,2024-09-03 15:27:37,683,F,...,5,277.28,Вторник,15,3,-0.707107,-0.707107,0.309017,0.951057,1
7463,7463,4548,20535841,14.6150,2225,2024-09-03 15:02:04,20.0,2024-09-03 15:17:18,707,F,...,2,21.05,Вторник,15,2,-0.707107,-0.707107,0.207912,0.978148,1
7464,7464,7099,20535837,12.1976,3354,2024-09-03 15:02:27,25.0,2024-09-03 15:27:40,683,F,...,5,102.11,Вторник,15,2,-0.707107,-0.707107,0.207912,0.978148,1
7465,7465,4348,20535835,49.2960,2129,2024-09-03 15:03:49,30.0,2024-09-03 15:20:34,683,F,...,3,43.43,Вторник,15,3,-0.707107,-0.707107,0.309017,0.951057,1


# Изменение дата сета

In [ ]:
df_data['order_start_prepare'] = pd.to_datetime(df_data['order_start_prepare'])
df_data['order_ready'] = pd.to_datetime(df_data['order_ready'])
df_data['hour_ready']=df_data['order_ready'].dt.hour
df_data['minutes_ready']=df_data['order_ready'].dt.minute
df_data.drop(columns=['day_of_week','region_name','region_id','store_name','Unnamed: 0','hour_start_sin','hour_start_cos','minutes_start_sin','order_start_prepare','minutes_start_cos','order_ready','status_id'], inplace=True)

In [ ]:
df_data

,store_id,order_id,profit,delivery_distance,planned_prep_time,items_count,order_price,hour_start,minutes_start,prepared_on_time,hour_ready,minutes_ready
0,5014,20562761,45.5040,2205,15.0,2,30.49,15,25,1,15,32
1,3856,20562753,30.3360,1368,10.0,2,240.78,15,25,1,15,35
2,10825,20562723,43.6080,459,20.0,1,85.99,15,23,1,15,38
3,5640,20562673,32.8640,2611,20.0,2,24.68,15,19,1,15,35
4,7283,20562665,11.0600,1041,15.0,1,14.02,15,14,0,15,35
...,...,...,...,...,...,...,...,...,...,...,...,...
7462,7557,20535845,50.1650,8046,35.0,5,277.28,15,3,1,15,27
7463,4548,20535841,14.6150,2225,20.0,2,21.05,15,2,1,15,17
7464,7099,20535837,12.1976,3354,25.0,5,102.11,15,2,1,15,27
7465,4348,20535835,49.2960,2129,30.0,3,43.43,15,3,1,15,20


In [ ]:
print (Counter(df_data['prepared_on_time']))

Counter({1: 6050, 0: 1417})


# Подбор гиперпараметров

In [ ]:
X = df_data.drop(columns=['prepared_on_time'])
y = df_data['prepared_on_time']

x_train1, x_test1, y_train1, y_test1 = train_test_split(X, y,test_size=0.3, random_state=42)
param_grid={
    'max_depth': list(range(6, 21)),
    'min_samples_leaf': range(1, 20, 2),
    'min_samples_split': range(2, 20, 2),
    'criterion': ["entropy", "gini"],
    'class_weight': [
        None,
        'balanced',
        {0: 6, 1: 1},
        {0: 4, 1: 1}
    ]
}
clf = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid,scoring='accuracy', cv=5, verbose=1)

# Обучение
grid_search.fit(x_train1, y_train1)

# Получение результатов
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Лучшие гиперпараметры: {best_params}')
print(f'Лучшая точность: {best_score}')

# Точность на тестовом наборе
best_clf = grid_search.best_estimator_
test_accuracy = best_clf.score(x_test1, y_test1)

print(f'Точность на тестовом наборе: {test_accuracy}')


Fitting 5 folds for each of 10800 candidates, totalling 54000 fits
Лучшие гиперпараметры: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 12, 'min_samples_leaf': 7, 'min_samples_split': 2}
Лучшая точность: 0.8103760966818229
Точность на тестовом наборе: 0.8112449799196787


# Визуализация/интерпритация данных

In [ ]:
fn = X.columns.tolist()
cn = y.unique().astype(str).tolist()
fig, ax = plt.subplots(figsize=(50, 20), dpi=500)
tree.plot_tree(
    best_clf,
    feature_names=fn,
    class_names=cn,
    filled=True,
    ax=ax,
    fontsize=10,
)
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
feature=X.columns
importances = best_clf.feature_importances_
for i, v in enumerate(importances):
    print(f"Признак {feature[i]}: Важность = {v:.4f}")

Признак store_id: Важность = 0.0640
Признак order_id: Важность = 0.0863
Признак profit: Важность = 0.0845
Признак delivery_distance: Важность = 0.0982
Признак planned_prep_time: Важность = 0.2027
Признак items_count: Важность = 0.0080
Признак order_price: Важность = 0.0416
Признак hour_start: Важность = 0.0176
Признак minutes_start: Важность = 0.2148
Признак hour_ready: Важность = 0.0218
Признак minutes_ready: Важность = 0.1605


Видно, что большую роль сыграли признаки planned_prep_time и minutes_start, minutes_ready, что неудивительно, ведь целевая переменная prepared_on_time зависит от времени. На удивление, признаки items_count и hour_start оказались бесполезными при построении дерева, но в целом это объяснимо. Время доставки могло зависеть только от большого количества предметов, а в нашем датасете максимум 12 предметов и таких значений немного.

In [ ]:
print (Counter(df_data['items_count']))
print (Counter(df_data['planned_prep_time']))

Counter({2: 2311, 1: 2039, 3: 1528, 4: 847, 5: 401, 6: 186, 7: 80, 8: 38, 9: 17, 10: 13, 11: 5, 12: 2})
Counter({20.0: 2172, 25.0: 1647, 15.0: 1410, 30.0: 972, 35.0: 491, 10.0: 424, 40.0: 180, 45.0: 65, 5.0: 49, 38.0: 12, 28.0: 8, 22.0: 5, 43.0: 4, 26.0: 4, 33.0: 3, 42.0: 3, 37.0: 3, 27.0: 3, 36.0: 2, 32.0: 2, 6.0: 2, 12.0: 2, 23.0: 1, 11.0: 1, 13.0: 1, 19.0: 1})


In [ ]:
clf1 = DecisionTreeClassifier(criterion = 'entropy',max_depth=12,min_samples_leaf= 7,min_samples_split= 2)
clf1.fit(x_train1, y_train1)
tree_pred_res = clf1.predict(x_test1)
tree_accuracy = accuracy_score(y_test1, tree_pred_res)
tree_classification_report=classification_report(y_test1, tree_pred_res)
tree_f1_score=f1_score(y_test1, tree_pred_res)
tree_precision=precision_score(y_test1, tree_pred_res)
tree_recall=recall_score(y_test1, tree_pred_res)
print(tree_classification_report)
print(f'accuracy: {tree_accuracy} \nprecision: {tree_precision} \nrecall: {tree_recall} \nf1-score: {tree_f1_score}')

              precision    recall  f1-score   support

           0       0.44      0.25      0.32       395
           1       0.85      0.93      0.89      1846

    accuracy                           0.81      2241
   macro avg       0.65      0.59      0.60      2241
weighted avg       0.78      0.81      0.79      2241

accuracy: 0.8116912092815707 
precision: 0.852824578790882 
recall: 0.9322860238353196 
f1-score: 0.8907867494824017


По метрикам можно сделать вывод что модель хорошо классифицирует класс 1 ,но у нее проблемы с классом 0,что не удивительно ведь классы не сбалансированы